Installing Tensorflow Lite model maker

In [1]:
!pip install -q tflite-model-maker

     |████████████████████████████████| 591 kB 5.4 MB/s 
     |████████████████████████████████| 120 kB 45.2 MB/s 
     |████████████████████████████████| 1.1 MB 32.8 MB/s 
     |████████████████████████████████| 840 kB 47.0 MB/s 
     |████████████████████████████████| 64 kB 2.3 MB/s 
     |████████████████████████████████| 6.3 MB 45.9 MB/s 
     |████████████████████████████████| 636 kB 46.5 MB/s 
     |████████████████████████████████| 1.2 MB 55.4 MB/s 
     |████████████████████████████████| 103 kB 53.1 MB/s 
     |████████████████████████████████| 211 kB 61.0 MB/s 
     |████████████████████████████████| 87 kB 5.0 MB/s 
     |████████████████████████████████| 1.1 MB 46.0 MB/s 
     |████████████████████████████████| 99 kB 6.7 MB/s 
     |████████████████████████████████| 352 kB 44.9 MB/s 
     |████████████████████████████████| 37.1 MB 44 kB/s 
     |████████████████████████████████| 200 kB 57.1 MB/s 


Importing Libraries

In [2]:
import numpy as np
from numpy.random import RandomState
import pandas as pd
import os
from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.text_classifier import AverageWordVecSpec
from tflite_model_maker.text_classifier import DataLoader
import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

Loading Train and Test Dataset

In [3]:
df_train = pd.read_csv('train.csv', error_bad_lines = False, engine = "python")
df_test = pd.read_csv('test.csv', error_bad_lines = False, engine = "python")

In [4]:
print(df_train.shape)
df_train.head()

(700, 3)


,Unnamed: 0,sentence,label
0,0,"wodaeeex3: I Want Snake Bites , MY Industrial ...",Offensive
1,1,What a wonderful thought it is that some of th...,Non-Offensive
2,2,VineForTheByrd: On my way to fuck your bitch L...,Offensive
3,3,__sneakyb: I hate when boys deny that they hav...,Offensive
4,4,Love yourself. It is important to stay positiv...,Non-Offensive


In [5]:
print(df_test.shape)
df_test.head()

(300, 3)


,Unnamed: 0,sentence,label
0,0,A7XDemery: I'm a fucking fag they said,Offensive
1,1,you dodge a bullet DaRealKha: All da bitches ...,Offensive
2,2,MadFlyentist: Oregon chokes every year get off...,Offensive
3,3,Linniekravitz: WesGodLaFlare bitch wtf,Offensive
4,4,"You have to be positive, and Im not just talki...",Non-offensive


Selecting Model

In [6]:
spec = model_spec.get('average_word_vec')
#spec = model_spec.get('mobilebert_classifier')
# spec = model_spec.get(‘bert_classifier’)
# spec = AverageWordVecSpec(wordvec_dim = 32)

In [7]:
#Only when using mobilebert_classifier
#spec.seq_len = 256

Loading Training and Test data

In [8]:
train_data = DataLoader.from_csv(
     filename = 'train.csv',
     text_column = 'sentence',
     label_column = 'label',
     model_spec = spec,
     is_training = True)

test_data = DataLoader.from_csv(
     filename = 'test.csv',
     text_column = 'sentence',
     label_column = 'label',
     model_spec = spec,
     is_training = False)

Training Model

In [9]:
model = text_classifier.create(train_data, model_spec = spec, epochs = 100)

Epoch 1/100
21/21 [==============================] - 1s 9ms/step - loss: 0.6934 - accuracy: 0.4673
Epoch 2/100
21/21 [==============================] - 0s 6ms/step - loss: 0.6928 - accuracy: 0.5193
Epoch 3/100
21/21 [==============================] - 0s 6ms/step - loss: 0.6925 - accuracy: 0.5804
Epoch 4/100
21/21 [==============================] - 0s 6ms/step - loss: 0.6917 - accuracy: 0.6116
Epoch 5/100
21/21 [==============================] - 0s 6ms/step - loss: 0.6908 - accuracy: 0.6696
Epoch 6/100
21/21 [==============================] - 0s 6ms/step - loss: 0.6900 - accuracy: 0.6875
Epoch 7/100
21/21 [==============================] - 0s 6ms/step - loss: 0.6881 - accuracy: 0.7396
Epoch 8/100
21/21 [==============================] - 0s 5ms/step - loss: 0.6841 - accuracy: 0.6875
Epoch 9/100
21/21 [==============================] - 0s 6ms/step - loss: 0.6820 - accuracy: 0.6964
Epoch 10/100
21/21 [==============================] - 0s 6ms/step - loss: 0.6768 - accuracy: 0.7827
Epoch 11/

Model Details

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 256, 16)           38064     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 34        
Total params: 38,370
Trainable params: 38,370
Non-trainable params: 0
_________________________________________________________________


Evaluating Model

In [11]:
loss, acc=model.evaluate(test_data)

10/10 [==============================] - 0s 9ms/step - loss: 0.0672 - accuracy: 0.9633


Exporting TF Lite model

In [12]:
model.export(export_dir='average_word_vec')